In [110]:
using Combinatorics

# Northwest Corner Method

In [331]:
function NCM(in_S, in_D, in_C)
    S, D, C = copy(in_S), copy(in_D), copy(in_C)
    
    #print demand/supply
    println("Total Supply: $(sum(S))", "\n", "Total Demand: $(sum(D))")
    
    #compare supply and demand
    if sum(S) > sum(D) 
        D = [D; sum(S)-sum(D)]
        C = [C zeros(size(S)[1])]
    elseif sum(S) < sum(D) 
        S = [S; sum(D)-sum(S)]
        C = [C; zeros(size(D)[1])']
    end
    
    #get m and n
    m, n = size(C)
    
    #get X vector
    X = fill(NaN, m, n)
    
    #start algorithm
    i, j = 1, 1
    while count(.!isnan.(X)) < m + n - 1
        x = min(S[i], D[j])
        X[i,j] = x
        if x == S[i]
            i += 1
            D[j] -= x
        else
            j += 1
            S[i] -= x
        end
    end
    
    return X
end

NCM (generic function with 1 method)

# Minimum-Cost Method

In [395]:
function MCM(in_S, in_D, in_C)
    S, D, C = copy(in_S), copy(in_D), copy(in_C)
    
    #print demand/supply
    println("Total Supply: $(sum(S))", "\n", "Total Demand: $(sum(D))")
    
    #compare supply and demand
    if sum(S) > sum(D) 
        D = [D; sum(S)-sum(D)]
        C = [C zeros(size(S)[1])]
    elseif sum(S) < sum(D) 
        S = [S; sum(D)-sum(S)]
        C = [C; zeros(size(D)[1])']
    end
    
    #get m and n
    m, n = size(C)
    
    #get X vector
    X = fill(NaN, m, n)
    
    #start algorithm
    while count(.!isnan.(X)) < m + n - 1
        #get min cost
        i, j = ind2sub(size(C),indmin(C))
        
        #get entry value
        x = min(S[i], D[j])
        X[i,j] = x
        
        #adjust supply/demand remaining and remove min cost
        if S[i] == D[j] 
            if count(.!isnan.(C[i,:])) >= count(.!isnan.(C[:,j]))
                S[i] -= x
                C[:,j] .= NaN
            else
                D[j] -= x
                C[i,:] .= NaN
            end
        elseif x == S[i]
            D[j] -= x
            C[i,:] .= NaN        
        else
            S[i] -= x
            C[:,j] .= NaN 
        end
    end
    
    return X
end

MCM (generic function with 1 method)

# Vogel Method

In [400]:
function Vogel(in_S, in_D, in_C)
    S, D, C = copy(in_S), copy(in_D), copy(in_C)
    
    #print demand/supply
    println("Total Supply: $(sum(S))", "\n", "Total Demand: $(sum(D))")
    
    #compare supply and demand
    if sum(S) > sum(D) 
        D = [D; sum(S)-sum(D)]
        C = [C zeros(size(S)[1])]
    elseif sum(S) < sum(D) 
        S = [S; sum(D)-sum(S)]
        C = [C; zeros(size(D)[1])']
    end
    
    #get m and n
    m, n = size(C)
    
    #get X vector
    X = fill(NaN, m, n)
    
    #start algorithm
    while count(.!isnan.(X)) < m + n - 1
        
        r_dif = sort(C, 2)[:,2]-sort(C, 2)[:,1]
        d_dif = sort(C, 1)[2,:]-sort(C, 1)[1,:]
                
        if count(.!isnan.(r_dif)) >= 1 && count(.!isnan.(d_dif)) >= 1
            if findmax(r_dif)[1] >= findmax(d_dif)[1]
                i = indmax(r_dif)
                j = indmin(C[i,:])
            else
                j = indmax(d_dif)
                i = indmin(C[:,j])
            end
        elseif count(.!isnan.(r_dif)) == 0 && count(.!isnan.(d_dif)) >= 1
            j = indmax(d_dif)
            i = indmin(C[:,j])
        elseif count(.!isnan.(d_dif)) == 0 && count(.!isnan.(r_dif)) >= 1
            i = indmax(r_dif)
            j = indmin(C[i,:])
        else
            i, j = ind2sub(size(C),indmin(C))
        end

        #get entry value
        x = min(S[i], D[j])
        X[i,j] = x
        
        #adjust supply/demand remaining and remove min cost
        if S[i] == D[j] 
            if count(.!isnan.(C[i,:])) >= count(.!isnan.(C[:,j]))
                S[i] -= x
                D[j] -= x
                C[:,j] .= NaN
            else
                S[i] -= x
                D[j] -= x
                C[i,:] .= NaN
            end
        elseif x == S[i]
            S[i] -= x
            D[j] -= x
            C[i,:] .= NaN        
        else
            S[i] -= x
            D[j] -= x
            C[:,j] .= NaN 
        end
    end
    
    return X
end

Vogel (generic function with 1 method)

# General Solver

In [408]:
function GeneralSolver(in_S, in_D, in_C)
    S, D, C = copy(in_S), copy(in_D), copy(in_C)
    
    #print demand/supply
    println("Total Supply: $(sum(S))", "\n", "Total Demand: $(sum(D))")
    
    #compare supply and demand
    if sum(S) > sum(D) 
        D = [D; sum(S)-sum(D)]
        C = [C zeros(size(S)[1])]
    elseif sum(S) < sum(D) 
        S = [S; sum(D)-sum(S)]
        C = [C; zeros(size(D)[1])']
    end
    
    #get m and n
    m, n = size(C)
    
    #define constraint table
    a1 = zeros(0)
    a2 = zeros(0)
    for i in 1:m
        Z = zeros(m,n)
        Z[i,:] .= 1
        a1 = [a1; Z'] 
        a2 = [a2; eye(n)]
    end
    
    A = [a1'; a2']
    b = [S; D]
    c = vec(C')'
    
    var_names = String[]
    for i in 1:m, j in 1:n
        push!(var_names, "x_$(i)$(j)")
    end
    
    A = A[2:end,:]
    b = b[2:end]

    #output
    output = Dict()
    
    counter = 0
    for basic_pair in collect(combinations(1:length(var_names), m+n-1))
        counter += 1
        
        B = zeros(0)
        for t in 1:m+n-1 
            B = [B; A[:,basic_pair[t]]'] 
        end
        B = B'
        
        if det(B) != 0
            x_B = inv(B)*b
            
            var_x = String[]
            for var_i in basic_pair push!(var_x, var_names[var_i]) end
            
            c_B = zeros(0)
            for t in 1:m+n-1 c_B = [c_B; c[basic_pair[t]]] end
            
            c_B = c_B'
            
            z = c_B*x_B
            
            if "Cost" in keys(output)
                if output["Cost"] > z && sum(abs.(x_B)) == sum(x_B)
                    output["Cost"] = z
                    output["Basic Pair"] = var_x
                    output["Value"] = x_B
                end
            elseif sum(abs.(x_B)) == sum(x_B)
                output["Cost"] = z
                output["Basic Pair"] = var_x
                output["Value"] = x_B 
            end
                      
        end
    end
    println("\n", "Iterations: $(counter)", "\n")
    return output, ["_" "_"; "Cost" output["Cost"]; "_" "_"; output["Basic Pair"] output["Value"]; "_" "_"]
end

GeneralSolver (generic function with 1 method)

# Exercises

### 2

In [284]:
ex2_S = [10, 10, 20]
ex2_D = [15, 10, 5, 10]
ex2_C = [4 6 9 2; 5 2 4 8; 7 4 9 12];

### 3

In [291]:
ex3_S = [50, 50]
ex3_D = [20, 20, 20]
ex3_C = [800 700 400; 600 800 500];

### 4

In [286]:
ex4_S = [400, 600, 400, 600, 1000]
ex4_D = [600, 1000, 800]
M = 10000; ex4_C = [31 29 32 28 29; 45 41 46 42 43; 38 35 40 copy(M) copy(M)]';

In [287]:
NCM(ex4_S, ex4_D, ex4_C)

Total Supply: 3000
Total Demand: 2400


5×4 Array{Float64,2}:
 400.0  NaN    NaN    NaN  
 200.0  400.0  NaN    NaN  
 NaN    400.0  NaN    NaN  
 NaN    200.0  400.0  NaN  
 NaN    NaN    400.0  600.0

In [288]:
MCM(ex4_S, ex4_D, ex4_C)

Total Supply: 3000
Total Demand: 2400


5×4 Array{Float64,2}:
 NaN     NaN    NaN    400.0
   0.0   NaN    400.0  200.0
 NaN     NaN    400.0  NaN  
 600.0   NaN    NaN    NaN  
 NaN    1000.0  NaN    200.0

In [289]:
Vogel(ex4_S, ex4_D, ex4_C)

Total Supply: 3000
Total Demand: 2400


5×4 Array{Float64,2}:
 NaN     NaN    200.0  200.0
 NaN     NaN    600.0  NaN  
 NaN     NaN    NaN    400.0
 600.0   NaN    NaN    NaN  
   0.0  1000.0  NaN      0.0

In [290]:
ex4_sol = GeneralSolver(ex4_S, ex4_D, ex4_C)

Total Supply: 3000
Total Demand: 2400

Iterations: 125970



Dict{Any,Any} with 3 entries:
  "Cost"       => 88400.0
  "Value"      => [0.0, 0.0, 200.0, 200.0, 600.0, 400.0, 600.0, 1000.0]
  "Basic Pair" => String["x_11", "x_12", "x_13", "x_14", "x_23", "x_34", "x_41"…

In [ ]:
ex4_sol[2]

### 5

In [353]:
function ex5()
    
    output = Dict()
    
    for Δ in 0:(sum([60, 80, 40])-sum([40, 60, 20]))
        ex5_S = [60, 80, 40]
        ex5_D = [40, 60, 80-Δ, 0+Δ]
        ex5_C = -[800 700 500 200; 500 200 100 300; 600 400 300 500]
        
        if "Cost" in keys(output)
            new_output = GeneralSolver(ex5_S, ex5_D, ex5_C)[1]
            if new_output["Cost"] < output["Cost"]
                output = new_output
            elseif new_output["Cost"] == output["Cost"]
                println(new_output["Value"])
                println(new_output["Basic Pair"])
            end
        else
            output = GeneralSolver(ex5_S, ex5_D, ex5_C)[1]
        end
    end
    
    return output
end

ex5 (generic function with 1 method)

_________

In [322]:
ex5()

Dict{Any,Any} with 3 entries:
  "Cost"       => -90000.0
  "Value"      => [0.0, 60.0, 40.0, 20.0, 20.0, 40.0]
  "Basic Pair" => String["x_11", "x_12", "x_21", "x_23", "x_24", "x_34"]

_________

### 6

In [335]:
ex6_S = [20, 16, 14, 20, 62]
ex6_D = [48, 60, 64, 28]
M = 10000; ex6_C = [400 450 350 copy(M); 1350 copy(M) 750 450; 850 500 900 550; 700 copy(M) 700 900; 450 650 500 1250];

In [336]:
NCM(ex6_S, ex6_D, ex6_C)

Total Supply: 132
Total Demand: 200


6×4 Array{Float64,2}:
  20.0  NaN    NaN    NaN  
  16.0  NaN    NaN    NaN  
  12.0    2.0  NaN    NaN  
 NaN     20.0  NaN    NaN  
 NaN     38.0   24.0  NaN  
 NaN    NaN     40.0   28.0

In [337]:
MCM(ex6_S, ex6_D, ex6_C)

Total Supply: 132
Total Demand: 200


6×4 Array{Float64,2}:
 NaN    NaN     20.0  NaN  
 NaN    NaN    NaN     16.0
 NaN     14.0  NaN    NaN  
 NaN      8.0  NaN     12.0
 NaN     18.0   44.0  NaN  
  48.0   20.0  NaN    NaN  

In [338]:
Vogel(ex6_S, ex6_D, ex6_C)

Total Supply: 132
Total Demand: 200


6×4 Array{Float64,2}:
 NaN    NaN     20.0  NaN  
 NaN    NaN    NaN     16.0
 NaN    NaN     10.0    4.0
 NaN    NaN     20.0  NaN  
  48.0  NaN     14.0  NaN  
 NaN     60.0  NaN      8.0

In [357]:
ex6_sol = GeneralSolver(ex6_S, ex6_D, ex6_C);

Total Supply: 132
Total Demand: 200

Iterations: 1307504



In [356]:
ex6_sol[2]

13×2 Array{Any,2}:
 "_"          "_"
 "Cost"  63800.0 
 "_"          "_"
 "x_13"     20.0 
 "x_24"     16.0 
 "x_32"     14.0 
 "x_43"     20.0 
 "x_51"     48.0 
 "x_53"     14.0 
 "x_62"     46.0 
 "x_63"     10.0 
 "x_64"     12.0 
 "_"          "_"

### 7

### 8

In [387]:
ex8_S = [1, 1, 1, 1]
ex8_D = [2, 1, 1, 1]
M = 1000; 
ex8_C = -1000*[85 110 -copy(M) -copy(M);
    93 113 125 76; 
    97 -copy(M) 116 59; 
    -copy(M) -copy(M) 124 72]';

In [397]:
NCM(ex8_S, ex8_D, ex8_C)

Total Supply: 4
Total Demand: 5


5×4 Array{Float64,2}:
   1.0  NaN    NaN    NaN  
   1.0  NaN    NaN    NaN  
   0.0    1.0  NaN    NaN  
 NaN      0.0    1.0  NaN  
 NaN    NaN      0.0    1.0

In [396]:
MCM(ex8_S, ex8_D, ex8_C)

Total Supply: 4
Total Demand: 5


5×4 Array{Float64,2}:
   0.0  NaN      1.0  NaN  
   1.0    0.0  NaN    NaN  
 NaN      1.0  NaN    NaN  
   0.0  NaN    NaN      1.0
   1.0  NaN    NaN    NaN  

In [403]:
Vogel(ex8_S, ex8_D, ex8_C)

Total Supply: 4
Total Demand: 5


5×4 Array{Float64,2}:
 NaN    NaN      1.0  NaN  
   1.0  NaN    NaN    NaN  
 NaN    NaN    NaN      1.0
 NaN      1.0    0.0    0.0
   1.0  NaN      0.0  NaN  

In [409]:
ex8_sol = GeneralSolver(ex8_S, ex8_D, ex8_C);

Total Supply: 4
Total Demand: 5

Iterations: 125970



In [410]:
ex8_sol[2]

12×2 Array{Any,2}:
 "_"            "_"
 "Cost"  -407000.0 
 "_"            "_"
 "x_11"        0.0 
 "x_12"        0.0 
 "x_13"        1.0 
 "x_14"        0.0 
 "x_21"        1.0 
 "x_34"        1.0 
 "x_42"        1.0 
 "x_51"        1.0 
 "_"            "_"